<h2>Assignment 1: Implementing Naive Bayes algorithm.</h2>
<ol>
    <li>The instructions for completing the assignment and the mark distribution is specified before each relevant section. Please make changes only in the area marked by the comment <i><b>#Write your code here</b></i>. Please do not change the code elsewhere.</li>
    <li>This assignment will be marked out of 7.</li>
    <li>Use of any external libraries like sklearn in your answers is prohibited.</li>
    <li>The students need to submit the .ipynb file and the corresponding .py file as part of the assignment. To get the corresponding .py file in the jupyter notebook, go to File > Download > Download .py.</li>
    <li>Please rename the .ipynb and .py files as FirstName_LastName_Assignment1.ipynb and FirstName_LastName_Assignment1.py, respectively.</li>
</ol>
<p><b>Task</b> : The task is to build a Naive Bayes classifier on a mixed dataset, i.e., the dataset contains both numerical and categorical features. You will be provided with a dataset stored in the variable <i>mydata</i>. Then, you have to calculate conditional probabilities and find the posterior probability of three samples stored in <i>data_eval</i> pandas dataframe.</p>

In [2]:
#Importing necessary libraries
import csv
import numpy as np
import pandas as pd
import math
!pip install wget
import wget
import copy
from os.path import exists

In [3]:
#Download the dataset .csv file from dropbox
if not exists("statlog_australian_data.csv"):
    wget.download("https://www.dropbox.com/s/zqey17e3d9tw1d5/statlog_australian_data.csv?dl=1", "statlog_australian_data.csv")
#Read the dataset in a pandas dataframe variable. The variable mydata stores the training set.
mydata = pd.read_csv('statlog_australian_data.csv')

dat_eval = [['a',22.67,7.000,'g','c','bb',0.165,'f','f',0,'f','g',160,1,0], \
            ['a',24.58,10.750,'g','j','bb',1.250,'t','f',0,'t','g',200,1,0], \
            ['a',21.67,11.500,'p','k','zz',0.000,'t','t',11,'t','g',0,1,1]]
#The variable mydata_eval stores the three examples on which we will test the Naïve Bayes Model.
mydata_eval = pd.DataFrame(dat_eval, columns=list(mydata.columns))
print('Number of samples: ',mydata.shape[0])
print('Number of features: ',mydata.shape[1]-1)
print('First 10 samples from data:')
mydata.loc[:10,:]


Number of samples:  690
Number of features:  14
First 10 samples from data:


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,b,22.08,11.460,g,j,bb,1.585,f,f,0,t,g,100,1213,0
1,a,22.67,7.000,g,c,bb,0.165,f,f,0,f,g,160,1,0
2,a,29.58,1.750,p,j,bb,1.250,f,f,0,t,g,280,1,0
3,a,21.67,11.500,p,k,j,0.000,t,t,11,t,g,0,1,1
4,b,20.17,8.170,g,aa,bb,1.960,t,t,14,f,g,60,159,1
5,a,15.83,0.585,g,c,h,1.500,t,t,2,f,g,100,1,1
6,b,17.42,6.500,g,i,bb,0.125,f,f,0,f,g,60,101,0
7,a,58.67,4.460,g,q,h,3.040,t,t,6,f,g,43,561,1
8,b,27.83,1.000,p,d,h,3.000,f,f,0,f,g,176,538,0
9,a,55.75,7.080,g,j,h,6.750,t,t,3,t,g,100,51,0


<h3>1. Scale the numerical features. [1 mark]</h3>
<p>The pandas dataframe variable <i><b>mydata</i></b> stores the data. The first 14 columns of the variable <i><b>mydata</i></b>, that is, <i><b>(A1 to A14)</i></b> are features and the last column <i><b>A15</i></b> refers to class labels. The data consists of both the numerical and categorical features.</p> 
<p>The numerical features need to be scaled. Please note that to scale the numerical features you cannot use 
sklearn.preprocessing.MinMaxScaler, sklearn.preprocessing.StandardScaler, or any other sklearn based scaler functions. Write your own scaler. Otherwise, you will lose a mark.</p>

In [4]:
# Check if there is any missing value in the dataset 
mydata.isna().sum().sort_values(ascending=False)

A15    0
A14    0
A13    0
A12    0
A11    0
A10    0
A9     0
A8     0
A7     0
A6     0
A5     0
A4     0
A3     0
A2     0
A1     0
dtype: int64

In [5]:
#Write your code here


# A2,A3,A7,A10,A13,A14 are numerical features 

# based on the histogram and box plot of the features, there are outliers in the data. 
# therefore, it is better to use the formula of standardization which works better on these type of data 

import matplotlib
import math

# to get the mean and std of each numercial column
def summary(feature):
   m=feature.mean(axis=0)
   std=feature.std(axis=0)
   return m,std

# apply standardization to each column
def transform(feature,m,std):
    std[std==0]=1
    arr = [[(feature[row][colum]- m[colum])/std[colum] for colum in range(feature.shape[1])] for row in range(feature.shape[0])]
    return arr
 
             
# separate the categorical and numerical features
numeric_features=mydata[['A2','A3','A7','A10','A13','A14']]
cat_features=mydata[['A1','A4','A5','A6','A8','A9','A11','A12','A15']]

# convert the numerical feeatures to numpy array 
numeric_features=numeric_features.to_numpy()
features_name=mydata.columns.to_list()

# find mean and std
m,std=summary(numeric_features)
# print(m,std)
label=mydata['A15']

# find the scaled version of features
new_features=transform(numeric_features,m,std)
# add labels back to numercial features 
new_num_features=np.c_[new_features,label]
# print(new_num_features)


<h3>2. Calculate conditional probabilities for categorical features. [1 mark]</h3>
<p>Calculate the conditional probabilities for categorical features and store in a variable.</p>

In [6]:
#Write your code here
# to see the frequency of labels in each feature 

# print(cat_features.loc[:5,:])
unique_counts = pd.DataFrame.from_records([(col, cat_features[col].nunique()) for col in cat_features.columns],
                           columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
print(unique_counts) 

  Column_Name  Num_Unique
0          A1           2
4          A8           2
5          A9           2
6         A11           2
8         A15           2
1          A4           3
7         A12           3
3          A6           8
2          A5          14


In [7]:
# finding th epivot table of each feature based on the A15 
def count_val(data, index, cols ):
   
    all_cols = index + cols
    counts = pd.pivot_table( data[all_cols] , index = index , columns= cols , aggfunc= 'size' ).replace(np.nan,0)
    return counts


def cond_prob_dist (probs,name,target,zero_freq):
    '''
    calculates the conditions prob. distribution where: row is A and column is B
    P(A | B) = P( A and B ) / P(B)

    This function is designed to create a new cond. prob. is there is a zero freq.
    probs = form function count_val
    name= the column name 
    target= the column that has zero freq, by default it is NONE
    zero_freq= False or True 

    It returns: a table with all elements cond prob. and the 0 and 1 counts for each column.

        '''
    CPD = probs.copy()
    col_totals = probs.sum(axis=0)
    mean=[]

    if zero_freq==False:
        # find the cond. prob.
        for col in col_totals.index:
            CPD[col] =   CPD[col] / col_totals.loc[col]
            mean.append(col_totals.loc[col])

    elif zero_freq==True:

        if name!=target:
            add=CPD.shape[0]
            for col in col_totals.index:
                c=CPD[col]+1
                d=col_totals.loc[col]+add
                # print(c)
                CPD[col] =  c / d
                mean.append(d)

        elif name==target:
            # since for the column a new element is introduced +1 is for that one
            add=CPD.shape[0]+1
            for col in col_totals.index:
                c=CPD[col]+1
                d=col_totals.loc[col]+add
                CPD[col] =   c/d
                # print(CPD[col])
                mean.append(d)
                # print(mean)
        
    CPD.columns = [ f'{x}' for i,x in enumerate(CPD.columns) ]
    CPD.index   = [ f'{name}-{x}' for i,x in enumerate(CPD.index) ] 
    return CPD.round(3), mean

# create a pandas datafame for the conditional probabilities 
def cond_prob_tabel(features, name, answer ):
   
    result=pd.DataFrame()
    num=[]
    for i in range(features.shape[1]-1):
        column_name=cat_features.columns
        JP=count_val(features,[column_name[i]],['A15'])
        new_val,m=cond_prob_dist(JP,column_name[i],name,answer)
        result=pd.concat([result,new_val])
        if name!='None':
            if name==column_name[i]:
                num=m
                print(num)
    return result,num
        
# finds the class prob. with respect to zero freq 
def class_prob(feature,answer,number):
    N=feature.shape[0]
    val=feature.value_counts()    
    if answer==True:
        z=val[0]+number
        o=val[1]+number
        N=N+(2*number)
        return z/N, o/N
    else:
        return val[0]/N, val[1]/N
        

# categorical features' conditional probability tabel 

cond_cat_tabel,s=cond_prob_tabel(cat_features,'None',False)
cond_cat_tabel




,0,1
A1-a,0.316,0.329
A1-b,0.684,0.671
A4-g,0.692,0.847
A4-gg,0.000,0.007
A4-p,0.308,0.147
A5-aa,0.091,0.062
A5-c,0.209,0.215
A5-cc,0.031,0.094
A5-d,0.060,0.023
A5-e,0.029,0.046


<h3>3. Calculate conditional probabilities for numerical features. [1 mark]</h3>
<p>For calculating conditional probabilities for numerical values, you can either discretize them first and then calculate conditional probabilities or use a gaussian distribution for calculating conditional probabilities.  You can also use any other method, such as kernel density estimation and calculate conditional probabilities.</p>
<ul>
 	<li>For discretization, you cannot use any standard sklearn library, e.g. sklearn.preprocessing.KBinsDiscretizer. Write your own scaler or you will lose mark.</li>
	<li>If you are using gaussian estimation/kernel density, then you cannot use sklearn.naive_bayes.GaussianNB, or you will lose mark.</li>
</ul>
<p><b>Write few sentences about your specific approach in the comments below.</b></p>

Describe your approach here:
1. I will start by separating the values based on their labels and save them in a dictionary with key values: 0 and 1.
2. I will find the mean and std of each feature based on their label. This part is the values for our model, which means the function calculates prob based on a class that will calculate probability of a row happening based on the dataset mean and std using gaussian probability 
3. After that I will calculate the gaussian probability



In [8]:
from math import sqrt
from math import pi
from math import exp
import statistics

# Split the dataset by class values, returns a dictionary
def separate_class(dataset):
	separated = {}
	for vector in dataset:
		class_value = vector[-1]
		if (class_value not in separated):
			separated[class_value] = []
		separated[class_value].append(vector)
	return separated
 

# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
	summaries = []
	for column in zip(*dataset):
		summaries.append([statistics.mean(column), statistics.stdev(column), len(column)])
	# print(summary)
	del(summaries[-1])
	return summaries
 
# Split dataset by class then calculate statistics for each row
def summarize_class(dataset):
	separated = separate_class(dataset)
	summaries = {}
	for class_value, rows in separated.items():
		summaries[class_value] = summarize_dataset(rows)
	return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
	exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
	return (1 / (sqrt(2 * pi) * stdev)) * exponent
 
# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
	rows=[]
	for label in summaries:
		rows.append(summaries[label][0][2])
	
	total_rows = rows[0]+rows[1]
	probabilities = {}
	for class_value, class_summaries in summaries.items():
		probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
		for i in range(len(class_summaries)):
			mean, stdev, _ = class_summaries[i]
			probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
	return probabilities

# we can say summaries is our model's weights
summaries = summarize_class(new_num_features)
print(summaries)

# calculate probability of rows from our dataset based on the summaries 
for i in range(new_num_features.shape[0]):
	probabilities = calculate_class_probabilities(summaries, new_num_features[i])
	print(f'Row {i}: {probabilities}')


{0.0: [[-0.1447040431406169, 0.9100365279760424, 383], [-0.18469530145573798, 0.8719699142344098, 383], [-0.2887129967987405, 0.6340984382166255, 383], [-0.3638598998510756, 0.3910036947904192, 383], [0.08950554744903767, 1.0457550141585517, 383], [-0.15726633248035704, 0.1289986193982924, 383]], 1.0: [[0.18052654241972763, 1.0780271083487336, 307], [0.2304179167998295, 1.0998944564266508, 307], [0.3601859210876798, 1.2322620905200292, 307], [0.45393596626371974, 1.3006178503718264, 307], [-0.11166327255042807, 0.9312077533060077, 307], [0.19619871446246498, 1.4712417635509498, 307]]}
Row 0: {0.0: 0.003572135976032237, 1.0: 0.0001727861416883}
Row 1: {0.0: 0.04009399331454053, 1.0: 0.000251315786162932}
Row 2: {0.0: 0.061485129897284926, 1.0: 0.00024358233315182232}
Row 3: {0.0: 1.974540368720607e-09, 1.0: 6.316219688763874e-05}
Row 4: {0.0: 4.268422683353243e-13, 1.0: 8.782420785196443e-05}
Row 5: {0.0: 0.01719237685494512, 1.0: 0.00011801224263684038}
Row 6: {0.0: 0.02158762002948555

<h3>4. Apply Bayes Theorem to find the probabilities of the test samples.</h3>
<p>The pandas dataframe variable <i><b>mydata_eval</i></b> contains the test samples.</p>

<h4>4a. Scale the numerical features of the test samples using the parameters of the training set. [1 mark]</h4>

In [9]:
#Write your code here

# add column name to data_eval
data_eval=pd.DataFrame(dat_eval, columns=['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15'])
print(data_eval)

# separate categorical and numerical featurs for further processing 
data_eval_cat=data_eval[['A1','A4','A5','A6','A8','A9','A11','A12','A15']].to_numpy()
data_eval_num=data_eval[['A2','A3','A7','A10','A13','A14']].to_numpy()

# data_eval_label=data_eval['A15'].to_numpy()
arr=[[0],[0],[1]]

# scale the test set
scaled_data_eval_num=transform(data_eval_num,m,std)

# add the label colume 
scaled_num=np.append(scaled_data_eval_num,arr,axis=1)
# scaled_num


  A1     A2     A3 A4 A5  A6     A7 A8 A9  A10 A11 A12  A13  A14  A15
0  a  22.67   7.00  g  c  bb  0.165  f  f    0   f   g  160    1    0
1  a  24.58  10.75  g  j  bb  1.250  t  f    0   t   g  200    1    0
2  a  21.67  11.50  p  k  zz  0.000  t  t   11   t   g    0    1    1


<h4>4b. Calculate the posterior probabilities and predict the class for three test samples. [3 marks]</h4>

In [10]:
#convert categorical table to dict
dict_tabel=cond_cat_tabel.to_dict()
# dict_tabel

In [11]:
# create categorical dataste that has all the rows in both keys: 0 and 1
new_data = {0:data_eval_cat,1:data_eval_cat}
print(new_data)


{0: array([['a', 'g', 'c', 'bb', 'f', 'f', 'f', 'g', 0],
       ['a', 'g', 'j', 'bb', 't', 'f', 't', 'g', 0],
       ['a', 'p', 'k', 'zz', 't', 't', 't', 'g', 1]], dtype=object), 1: array([['a', 'g', 'c', 'bb', 'f', 'f', 'f', 'g', 0],
       ['a', 'g', 'j', 'bb', 't', 'f', 't', 'g', 0],
       ['a', 'p', 'k', 'zz', 't', 't', 't', 'g', 1]], dtype=object)}


In [12]:
# a function to find zero freq element and its respective column
def search(num1,num2):
    map_arr =['A1','A4','A5','A6','A8','A9','A11','A12','A15']
    # final_dict = {}
    for key, value in num2.items():
        nested_dict = num1[str(key)]
        final_dict= []
        for index, x in enumerate(value):
            localarr = []
            # print(index)
            for index_y in range(len(x)-1):
                y=x[index_y]
                # print(index_y)
                if(nested_dict.get(f'{map_arr[index_y]}-{y}')==None):
                    localarr.append(f'{map_arr[index_y]}')     
                    localarr.append(f'{y}')     
            # final_dict[key].append(localarr)        
    return localarr


class_zero_freq_name=search(dict_tabel,new_data)
# create a new cond. prob table with adding value 1 for zz and adding 1 to the each element's count
zero_freq_new_cat_table,class_num=cond_prob_tabel(cat_features,class_zero_freq_name[0],True)

# convert the table to dict format
dict_new_cat=zero_freq_new_cat_table.to_dict()
zero_freq_new_cat_table


[392, 316]


,0,1
A1-a,0.317,0.330
A1-b,0.683,0.670
A4-g,0.689,0.842
A4-gg,0.003,0.010
A4-p,0.308,0.148
A5-aa,0.091,0.062
A5-c,0.204,0.209
A5-cc,0.033,0.093
A5-d,0.060,0.025
A5-e,0.030,0.047


In [13]:
# calculates the prob of the new element 
def cal_prob_zero(val):
    total=[]
    for i in range(len(val)):
        p=1/val[i]
        total.append(p.round(4))
    return(total)


def dict_prob_value(num1,num2,num3,answer,total,row):
    '''
    matches the categorical element of each row with the excited dicts 

    num1=no zero freq table
    num2= evval categorical data
    num3= zero freq table 
    answer= True or False 
    total=the new elements column count of 0 and 1
    row=which row of eval data has the new element 

    '''
    map_arr =['A1','A4','A5','A6','A8','A9','A11','A12','A15']
    final_dict = {}
    if answer==True: 
        class_prob=cal_prob_zero(total)

    for key, value in num2.items():
        nested_dict1= num1[str(key)]
        nested_dict2= num3[str(key)]

        final_dict[key] = []
        for index, x in enumerate(value):
            localarr = []
            for index_y in range(len(x)-1):
                y=x[index_y]
                if answer==True: 
                    if index!=row:
                        if(nested_dict1.get(f'{map_arr[index_y]}-{y}')!=None):
                            localarr.append(nested_dict1[f'{map_arr[index_y]}-{y}'])
                        else:
                            localarr.append(0)
                            # print(index,y,index_y,f'{map_arr[index_y]}-{y}')
                    else:
                        if(nested_dict2.get(f'{map_arr[index_y]}-{y}')!=None):
                            localarr.append(nested_dict2[f'{map_arr[index_y]}-{y}'])
                        else:
                            localarr.append(class_prob[key])
                            # print(index,y,index_y,f'{map_arr[index_y]}-{y}')
                else:
                    if(nested_dict1.get(f'{map_arr[index_y]}-{y}')!=None):
                            localarr.append(nested_dict1[f'{map_arr[index_y]}-{y}'])
                    else:
                        localarr.append(0)
                        # print(index,y,index_y,f'{map_arr[index_y]}-{y}')

            final_dict[key].append(localarr)
            # print(final_dict)
    return final_dict

# finds P(yes|Ai) and P(no|Ai)
def class_cat_prob(model,label,answer,row,number):
    local_dict={}
    zero1,one1=class_prob(label,False,number)
    zero2,one2=class_prob(label,True,number)

    for key, value in model.items():
        local_dict[key]=[]
        for index, x in enumerate(value):
            sum=1
            for index_y, y in enumerate(x):
                sum=y*sum   
            if answer==True:
                if key==0:
                    if index!=row:
                        local_dict[key].append(sum*zero1)
                    else:
                        local_dict[key].append(sum*zero2)

                else:
                    if index!=row:
                        local_dict[key].append(sum*one1)
                    else:
                        local_dict[key].append(sum*one2)
            if answer==False:
                if key==0:  
                    local_dict[key].append(sum*zero1)
                else:
                    local_dict[key].append(sum*one1)
                   
    return local_dict

# finds each elements cond. prob. for data_eval_cat
model_cat=dict_prob_value(dict_tabel,new_data, dict_new_cat,True,class_num,2)
# print(model_cat)

# finds the final P(yes|Ai) and P(no|Ai)
pred_cat=class_cat_prob(model_cat,mydata['A15'],True,2,1)
print(pred_cat)

# the original numerical data mean,std, and column length 
model_num=summaries
# print(model_num)


{0: [0.004735234536831347, 0.00044149358679824964, 5.0387222034585e-08], 1: [0.00017623047071110384, 0.00042243170692804954, 2.3142698750547412e-07]}


In [15]:
def predict(row):
	# probabilities = calculate_class_probabilities(summaries, row)
	best_label, best_prob = None, -1
	for class_value, probability in row.items():
		if best_label is None or probability > best_prob:
			best_prob = probability
			best_label = class_value
	return best_label

# multiplies the numerical and categorical probabilities
def final_prob(cat,num):
    arr=[]
    for i in range(len(num)):
        probabilities = calculate_class_probabilities(model_num,scaled_num[i])
        # key,value=probabilities.items()
        arr.append(probabilities)

    new={}
    for key,value in cat.items():
        new[key]=[]
        localarr=[]
        for index,x in enumerate(value):
            new[key].append(x*arr[index][key])
    return new
  
  
new=final_prob(pred_cat,scaled_num)
row1={0:new[0][0],1:new[1][0]}
row2={0:new[0][1],1:new[1][1]}
row3={0:new[0][2],1:new[1][2]}
print(f'Final Prediction:\nRow 1: class {predict(row1)}\nRow 2: class {predict(row2)}\nRow 3: class {predict(row3)}')
 


Final Prediction:
Row 1: class 0
Row 2: class 0
Row 3: class 1
